# Genetic selection of different MA kinds

In [1]:
import pandas as pd
import numpy as np
from simple.plotly import chartParallel, interactTable
from simple.funcs import vwap, cwma, epma
from simple.jurik import JMA
from talib import KAMA, TSF
from simple.geneopt import GeneOpt
from multiprocessing import current_process
from simple.backtest import getLong, getShort, getProfit, npBacktestLimit

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
# declare chart linestyles
line_styles = {
    'Tick': dict(color='gray', opacity=0.25),
    'Center': dict(color='blue', opacity=0.5),
    'qA': dict(color='red', opacity=0.5, dash='dot'),
    'qB': dict(color='green', opacity=0.5, dash='dot'),
    'Profit': dict(color='gray', width=3, opacity=0.4, secondary_y=True, shape='hv', connectgaps=True),
    'Buy': dict(mode='markers', color='green', symbol='triangle-up', size=10, line=dict(color="darkgreen", width=1)),
    'Sell': dict(mode='markers', color='red', symbol='triangle-down', size=10, line=dict(color="darkred", width=1))
}

In [4]:
def model(Period: int = (1000, 50000), StdDev: float = (1, 4, 0.1), Kind: int = (0,6,1)):
    if Period == 0 or StdDev == 0: return 0, {}
    Tick = T.Price

    if Kind == 0:
        Center = vwap(T, Period)
    elif Kind == 1:
        Center = JMA(T.Price, Period)
    elif Kind == 2:
        Center = KAMA(T.Price, Period)
    elif Kind == 3:
        Center = TSF(T.Price, Period)
    elif Kind == 4:
        Center = cwma(T.Price, Period)
    elif Kind == 5:
        Center = epma(T.Price, Period)

    std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + std * StdDev
    qB = Center - std * StdDev

    trades = npBacktestLimit(T, qA, qB)
    Buy, Sell = getLong(trades), getShort(trades)
    P = getProfit(trades)
    Profit = {'x': P.Index, 'y': P.Profit.cumsum()}

    return P.Profit.sum(), {
        'Count': len(P),
        'AvgProfit': P.Profit.mean() if len(P) > 0 else 0,
        'Sharpe': P.Profit.sum() / P.Profit.std() if len(P) > 1 else 0
     } if current_process().daemon else locals()

In [5]:
# Genetic optimizer
G = GeneOpt(model)
P = G.maximize(population_size=1024, generations=5)
model(**P)[0], P

  0%|          | 0/5 [00:00<?, ?it/s]

(8186.596788843071, {'Period': 33190, 'StdDev': 2.682271004811449, 'Kind': 4})

In [6]:
# Genetic result browser
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Fitness')

interactTable(model, X, height=600, **line_styles)

In [7]:
chartParallel(X)

FigureWidget({
    'data': [{'dimensions': [{'label': 'Period', 'values': array([ 1034,  1491,  1327, ..., 195…